In [1]:
import pandas as pd

df_ = pd.read_csv('c:/Users/chanyoung/Desktop/young_workspace/ba_module1/ML_pjt/BANK_pjt/data_for_bank_all.csv',index_col=0)
# df_.index.name = 'Date'

# 결과 확인
df_

,Open,High,Low,Close,Volume,log_return,target,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,base,cofix_short,exchange
Date,,,,,,,,,,,,,,,,
2014-12-23,7290.0,7290.0,7250.0,7290.0,2424.0,0.002060,1,7252.0,7275.5,7511.166667,7756.750000,8018.000000,7917.687500,2.0,2.00,1102.7
2014-12-24,7300.0,7365.0,7265.0,7365.0,21753.0,0.010236,1,7269.0,7264.5,7491.000000,7741.333333,8015.125000,7912.687500,2.0,2.00,1102.6
2014-12-26,7400.0,7420.0,7360.0,7385.0,7384.0,0.002712,1,7292.0,7271.0,7472.333333,7726.416667,8013.458333,7909.062500,2.0,2.00,1098.7
2014-12-29,7465.0,7465.0,7320.0,7340.0,10208.0,-0.006112,0,7319.0,7284.0,7454.666667,7713.583333,8011.916667,7905.520833,2.0,2.05,1097.8
2014-12-30,7375.0,7375.0,7110.0,7135.0,30329.0,-0.028327,0,7331.0,7287.5,7441.666667,7703.583333,8010.083333,7902.104167,2.0,2.05,1099.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,8190.0,8205.0,8010.0,8055.0,886870.0,-0.016621,0,8381.0,8471.5,8218.333333,7572.583333,6988.500000,6537.666667,3.5,3.57,1349.4
2024-04-02,8005.0,8085.0,7925.0,8025.0,978698.0,-0.003731,0,8275.0,8423.0,8224.833333,7597.583333,7004.000000,6545.458333,3.5,3.57,1352.1
2024-04-03,8000.0,8035.0,7870.0,7875.0,783412.0,-0.018868,0,8174.0,8389.0,8224.000000,7623.750000,7017.708333,6553.375000,3.5,3.57,1348.9


In [2]:
#타겟변수 따로 저장해두기
target = df_[['target']]
target 

,target
Date,
2014-12-23,1
2014-12-24,1
2014-12-26,1
2014-12-29,0
2014-12-30,0
...,...
2024-04-01,0
2024-04-02,0
2024-04-03,0


In [3]:
#log_return 띠기
import numpy as np
df_ = df_.drop(columns=["log_return"])

In [4]:
df_

,Open,High,Low,Close,Volume,target,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,base,cofix_short,exchange
Date,,,,,,,,,,,,,,,
2014-12-23,7290.0,7290.0,7250.0,7290.0,2424.0,1,7252.0,7275.5,7511.166667,7756.750000,8018.000000,7917.687500,2.0,2.00,1102.7
2014-12-24,7300.0,7365.0,7265.0,7365.0,21753.0,1,7269.0,7264.5,7491.000000,7741.333333,8015.125000,7912.687500,2.0,2.00,1102.6
2014-12-26,7400.0,7420.0,7360.0,7385.0,7384.0,1,7292.0,7271.0,7472.333333,7726.416667,8013.458333,7909.062500,2.0,2.00,1098.7
2014-12-29,7465.0,7465.0,7320.0,7340.0,10208.0,0,7319.0,7284.0,7454.666667,7713.583333,8011.916667,7905.520833,2.0,2.05,1097.8
2014-12-30,7375.0,7375.0,7110.0,7135.0,30329.0,0,7331.0,7287.5,7441.666667,7703.583333,8010.083333,7902.104167,2.0,2.05,1099.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,8190.0,8205.0,8010.0,8055.0,886870.0,0,8381.0,8471.5,8218.333333,7572.583333,6988.500000,6537.666667,3.5,3.57,1349.4
2024-04-02,8005.0,8085.0,7925.0,8025.0,978698.0,0,8275.0,8423.0,8224.833333,7597.583333,7004.000000,6545.458333,3.5,3.57,1352.1
2024-04-03,8000.0,8035.0,7870.0,7875.0,783412.0,0,8174.0,8389.0,8224.000000,7623.750000,7017.708333,6553.375000,3.5,3.57,1348.9


In [5]:
### log_return (로그 변동률) 생성
df_log = np.log(df_/df_.shift(1))

c:\Users\chanyoung\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [6]:
df_log

,Open,High,Low,Close,Volume,target,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,base,cofix_short,exchange
Date,,,,,,,,,,,,,,,
2014-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-24,0.001371,0.010236,0.002067,0.010236,2.194333,0.0,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632,0.0,0.000000,-0.000091
2014-12-26,0.013606,0.007440,0.012992,0.002712,-1.080436,0.0,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,0.0,0.000000,-0.003543
2014-12-29,0.008745,0.006046,-0.005450,-0.006112,0.323856,-inf,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,0.0,0.024693,-0.000819
2014-12-30,-0.012130,-0.012130,-0.029108,-0.028327,1.088933,NaN,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0.0,0.000000,0.001365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,-0.016621,0.174036,NaN,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336,0.0,-0.008368,0.001632
2024-04-02,-0.022848,-0.014733,-0.010668,-0.003731,0.098525,NaN,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191,0.0,0.000000,0.001999
2024-04-03,-0.000625,-0.006203,-0.006964,-0.018868,-0.222564,NaN,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209,0.0,0.000000,-0.002369


In [7]:
df_log.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', 'moving_average_5',
       'moving_average_10', 'moving_average_30', 'moving_average_60',
       'moving_average_120', 'moving_average_240', 'base', 'cofix_short',
       'exchange'],
      dtype='object')

In [8]:
df_log = df_log[['Open', 'High', 'Low', 'Close', 'Volume','moving_average_5',
       'moving_average_10', 'moving_average_30', 'moving_average_60',
       'moving_average_120', 'moving_average_240']]

In [9]:
df_log

,Open,High,Low,Close,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240
Date,,,,,,,,,,,
2014-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-24,0.001371,0.010236,0.002067,0.010236,2.194333,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632
2014-12-26,0.013606,0.007440,0.012992,0.002712,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458
2014-12-29,0.008745,0.006046,-0.005450,-0.006112,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448
2014-12-30,-0.012130,-0.012130,-0.029108,-0.028327,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,-0.016621,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336
2024-04-02,-0.022848,-0.014733,-0.010668,-0.003731,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191
2024-04-03,-0.000625,-0.006203,-0.006964,-0.018868,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209


In [10]:
df_log.isna().sum()

Open                  1
High                  1
Low                   1
Close                 1
Volume                1
moving_average_5      1
moving_average_10     1
moving_average_30     1
moving_average_60     1
moving_average_120    1
moving_average_240    1
dtype: int64

In [11]:
df_log = df_log.dropna()

In [12]:
df_log

,Open,High,Low,Close,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240
Date,,,,,,,,,,,
2014-12-24,0.001371,0.010236,0.002067,0.010236,2.194333,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632
2014-12-26,0.013606,0.007440,0.012992,0.002712,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458
2014-12-29,0.008745,0.006046,-0.005450,-0.006112,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448
2014-12-30,-0.012130,-0.012130,-0.029108,-0.028327,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432
2015-01-02,-0.034486,-0.032383,-0.007766,-0.009859,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,-0.016621,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336
2024-04-02,-0.022848,-0.014733,-0.010668,-0.003731,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191
2024-04-03,-0.000625,-0.006203,-0.006964,-0.018868,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209


In [13]:
df_log.to_csv('df_log.csv')

In [14]:
df_log_target = pd.merge(df_log, target, on='Date', how='inner')

In [15]:
df_log_target = df_log_target.drop(columns=["Close"])

In [16]:
df_log_target
#2284 rows × 12 columns

,Open,High,Low,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,target
Date,,,,,,,,,,,
2014-12-24,0.001371,0.010236,0.002067,2.194333,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632,1
2014-12-26,0.013606,0.007440,0.012992,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,1
2014-12-29,0.008745,0.006046,-0.005450,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,0
2014-12-30,-0.012130,-0.012130,-0.029108,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0
2015-01-02,-0.034486,-0.032383,-0.007766,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336,0
2024-04-02,-0.022848,-0.014733,-0.010668,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191,0
2024-04-03,-0.000625,-0.006203,-0.006964,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209,0


In [27]:
df_log_target.to_csv('df_log_target.csv')

In [65]:
df_log_target

,Open,High,Low,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,target
Date,,,,,,,,,,,
2014-12-24,0.001371,0.010236,0.002067,2.194333,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632,1
2014-12-26,0.013606,0.007440,0.012992,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,1
2014-12-29,0.008745,0.006046,-0.005450,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,0
2014-12-30,-0.012130,-0.012130,-0.029108,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0
2015-01-02,-0.034486,-0.032383,-0.007766,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336,0
2024-04-02,-0.022848,-0.014733,-0.010668,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191,0
2024-04-03,-0.000625,-0.006203,-0.006964,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209,0


In [17]:
#미래y예측값 설정을 위해
df_log_target['target'] = df_log_target['target'].shift(1)

In [18]:
df_log_target = df_log_target.dropna()

In [19]:
df_log_target #2283 rows × 11 columns

,Open,High,Low,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,target
Date,,,,,,,,,,,
2014-12-26,0.013606,0.007440,0.012992,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,1.0
2014-12-29,0.008745,0.006046,-0.005450,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,1.0
2014-12-30,-0.012130,-0.012130,-0.029108,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0.0
2015-01-02,-0.034486,-0.032383,-0.007766,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559,0.0
2015-01-05,-0.010582,-0.010560,-0.018599,1.353097,-0.006181,-0.001926,-0.002991,-0.001996,-0.000494,-0.000646,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336,0.0
2024-04-02,-0.022848,-0.014733,-0.010668,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191,0.0
2024-04-03,-0.000625,-0.006203,-0.006964,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209,0.0


In [20]:
df_log_target['target'] = df_log_target['target'].values.astype(int)

C:\Users\chanyoung\AppData\Local\Temp\ipykernel_41800\1730821277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_log_target['target'] = df_log_target['target'].values.astype(int)


In [21]:
df_log_target

,Open,High,Low,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,target
Date,,,,,,,,,,,
2014-12-26,0.013606,0.007440,0.012992,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,1
2014-12-29,0.008745,0.006046,-0.005450,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,1
2014-12-30,-0.012130,-0.012130,-0.029108,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0
2015-01-02,-0.034486,-0.032383,-0.007766,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559,0
2015-01-05,-0.010582,-0.010560,-0.018599,1.353097,-0.006181,-0.001926,-0.002991,-0.001996,-0.000494,-0.000646,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336,0
2024-04-02,-0.022848,-0.014733,-0.010668,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191,0
2024-04-03,-0.000625,-0.006203,-0.006964,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209,0


In [22]:
#라이브러리 호출
import pandas as pd
import numpy as np

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


# 단일

In [46]:
df_log['Close']

Date
2014-12-24    0.010236
2014-12-26    0.002712
2014-12-29   -0.006112
2014-12-30   -0.028327
2015-01-02   -0.009859
                ...   
2024-04-01   -0.016621
2024-04-02   -0.003731
2024-04-03   -0.018868
2024-04-04    0.023220
2024-04-05   -0.009349
Name: Close, Length: 2284, dtype: float64

<b/>하나의 변수</b>에 대한 시계열 데이터를 분석하는 모델은 기본적으로 AR, MA, ARMA, ARIMA 모델

In [47]:
## 데이터를 학습 세트와 테스트 세트로 분리
y = df_log['Close']
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42,shuffle=False)

In [50]:
ar_model = AutoReg(y_train, lags=1)

c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [51]:
# AR(p) 모델 학습
ar_model_fitted = ar_model.fit()

In [52]:
# 예측 및 성능 평가
y_pred_ar = ar_model_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [53]:
metrics = {
    "AR(p)": {
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_ar)),
        "MAE": mean_absolute_error(y_test, y_pred_ar)
}}
metrics

#{'AR(p)': {'RMSE': 0.015714823439810604, 'MAE': 0.01162912891120239}}

{'AR(p)': {'RMSE': 0.015714823439810604, 'MAE': 0.01162912891120239}}

### 슬라이딩 윈도우 10 넣어서 훈련 예측

In [70]:
# 데이터 프레임에서'Close'만을 특징으로 사용
features = df_log['Close']
features

Date
2014-12-24    0.010236
2014-12-26    0.002712
2014-12-29   -0.006112
2014-12-30   -0.028327
2015-01-02   -0.009859
                ...   
2024-04-01   -0.016621
2024-04-02   -0.003731
2024-04-03   -0.018868
2024-04-04    0.023220
2024-04-05   -0.009349
Name: Close, Length: 2284, dtype: float64

In [75]:
df_log

,Open,High,Low,Close,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240
Date,,,,,,,,,,,
2014-12-24,0.001371,0.010236,0.002067,0.010236,2.194333,0.002341,-0.001513,-0.002689,-0.001989,-0.000359,-0.000632
2014-12-26,0.013606,0.007440,0.012992,0.002712,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458
2014-12-29,0.008745,0.006046,-0.005450,-0.006112,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448
2014-12-30,-0.012130,-0.012130,-0.029108,-0.028327,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432
2015-01-02,-0.034486,-0.032383,-0.007766,-0.009859,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.017550,-0.019313,-0.019778,-0.016621,0.174036,-0.012215,-0.005591,0.002355,0.003296,0.002340,0.001336
2024-04-02,-0.022848,-0.014733,-0.010668,-0.003731,0.098525,-0.012728,-0.005742,0.000791,0.003296,0.002215,0.001191
2024-04-03,-0.000625,-0.006203,-0.006964,-0.018868,-0.222564,-0.012281,-0.004045,-0.000101,0.003438,0.001955,0.001209


In [80]:
from sklearn.model_selection import train_test_split

# 데이터 프레임에서'Close'만을 특징으로 사용
features = df_log['Close']

# 10일치 데이터로 구성된 슬라이딩 윈도우 생성
X = []
y = []
window_size = 10

for i in range(len(features) - window_size):
    X.append(features.iloc[i:i+window_size].values.flatten())  # 10일치 데이터를 하나의 배열로 펼침
    y.append(df_log.iloc[i + window_size]['Close'])  # 해당 윈도우 다음 날의 Close



In [82]:
# 배열로 변환
X = np.array(X)
y = np.array(y)

X

array([[ 0.0102355 ,  0.00271187, -0.00611207, ..., -0.01021907,
         0.03461052,  0.01337577],
       [ 0.00271187, -0.00611207, -0.02832659, ...,  0.03461052,
         0.01337577, -0.02693288],
       [-0.00611207, -0.02832659, -0.00985923, ...,  0.01337577,
        -0.02693288,  0.00143575],
       ...,
       [ 0.01365412,  0.02790879, -0.00172166, ..., -0.01754962,
        -0.01662088, -0.00373135],
       [ 0.02790879, -0.00172166, -0.01388108, ..., -0.01662088,
        -0.00373135, -0.01886848],
       [-0.00172166, -0.01388108, -0.00642713, ..., -0.00373135,
        -0.01886848,  0.02322037]])

In [83]:
y

array([-0.02693288,  0.00143575,  0.00785999, ..., -0.01886848,
        0.02322037, -0.00934877])

In [84]:
# 데이터 세트 분할: 훈련 세트 80%, 테스트 세트 20%
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42,shuffle= True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1819, 10), (455, 10), (1819,), (455,))

In [85]:
ar_model = AutoReg(y_train, lags=1)

In [86]:
# 슬라이딩 적용시킨 AR(p) 모델 학습
ar_model_fitted = ar_model.fit()

In [87]:
# 예측 및 성능 평가
y_pred_ar = ar_model_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

In [88]:
metrics = {
    "AR(p)": {
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_ar)),
        "MAE": mean_absolute_error(y_test, y_pred_ar)
}}
metrics

#{'AR(p)': {'RMSE': 0.014928575357944958, 'MAE': 0.011204520899578862}}

{'AR(p)': {'RMSE': 0.014928575357944958, 'MAE': 0.011204520899578862}}

# 다중

In [54]:
# 데이터를 학습 세트와 테스트 세트로 분리

# 데이터셋 분할 ->shuffle=False를 줘야함! 그래야 밑둥부터
X = df_log.drop(columns=["Close"])
y = df_log["Close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)

In [55]:
# 모델 정의
ar_model = AutoReg(y_train, lags=1)

c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [56]:
# AR(p) 모델 학습
ar_model_fitted = ar_model.fit()

In [57]:
# 예측 및 성능 평가
y_pred_ar = ar_model_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\chanyoung\anaconda3\Lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [60]:
metrics = {
    "AR(p)": {
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_ar)),
        "MAE": mean_absolute_error(y_test, y_pred_ar)
}}
metrics

# {'AR(p)': {'RMSE': 0.015714823439810604, 'MAE': 0.01162912891120239}}
# rmse와 mae가 낮을 수록 좋은 

{'AR(p)': {'RMSE': 0.015714823439810604, 'MAE': 0.01162912891120239}}

In [59]:
y_test, y_pred_ar

(Date
 2022-05-31    0.001301
 2022-06-02   -0.025685
 2022-06-03    0.002665
 2022-06-07   -0.014071
 2022-06-08    0.003368
                 ...   
 2024-04-01   -0.016621
 2024-04-02   -0.003731
 2024-04-03   -0.018868
 2024-04-04    0.023220
 2024-04-05   -0.009349
 Name: Close, Length: 457, dtype: float64,
 1827   -0.000232
 1828    0.000034
 1829    0.000023
 1830    0.000023
 1831    0.000023
           ...   
 2279    0.000023
 2280    0.000023
 2281    0.000023
 2282    0.000023
 2283    0.000023
 Length: 457, dtype: float64)

# 분류모형

In [28]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix

In [29]:
# XGBoost 모델
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_cm = confusion_matrix(y_test, xgb_pred)

In [30]:
# LightGBM 모델
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_pred = lgbm_model.predict(X_test)
lgbm_cm = confusion_matrix(y_test, lgbm_pred)

[LightGBM] [Info] Number of positive: 890, number of negative: 936
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 1826, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487404 -> initscore=-0.050394
[LightGBM] [Info] Start training from score -0.050394


In [31]:
# CatBoost 모델
cat_model = CatBoostClassifier(verbose=0)  # verbose=0 to avoid too much logging
cat_model.fit(X_train, y_train)
cat_pred = cat_model.predict(X_test)
cat_cm = confusion_matrix(y_test, cat_pred)

In [32]:
xgb_cm, lgbm_cm, cat_cm

(array([[179,  53],
        [ 56, 169]], dtype=int64),
 array([[182,  50],
        [ 61, 164]], dtype=int64),
 array([[184,  48],
        [ 61, 164]], dtype=int64))

In [33]:
from sklearn.metrics import classification_report

# XGBoost 성능 평가
report_xgb = classification_report(y_test, xgb_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# LightGBM 성능 평가
report_lgb = classification_report(y_test, lgbm_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# CatBoost 성능 평가
report_cb = classification_report(y_test, cat_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# 결과 출력
(report_xgb, report_lgb, report_cb)

({'Class 0': {'precision': 0.7617021276595745,
   'recall': 0.771551724137931,
   'f1-score': 0.7665952890792291,
   'support': 232},
  'Class 1': {'precision': 0.7612612612612613,
   'recall': 0.7511111111111111,
   'f1-score': 0.756152125279642,
   'support': 225},
  'accuracy': 0.7614879649890591,
  'macro avg': {'precision': 0.7614816944604179,
   'recall': 0.7613314176245211,
   'f1-score': 0.7613737071794355,
   'support': 457},
  'weighted avg': {'precision': 0.7614850708989169,
   'recall': 0.7614879649890591,
   'f1-score': 0.761453687646172,
   'support': 457}},
 {'Class 0': {'precision': 0.7489711934156379,
   'recall': 0.7844827586206896,
   'f1-score': 0.7663157894736842,
   'support': 232},
  'Class 1': {'precision': 0.7663551401869159,
   'recall': 0.7288888888888889,
   'f1-score': 0.7471526195899771,
   'support': 225},
  'accuracy': 0.7571115973741794,
  'macro avg': {'precision': 0.7576631668012769,
   'recall': 0.7566858237547893,
   'f1-score': 0.7567342045318306,


In [34]:
# XGBoost 성능 평가를 텍스트로 출력
print("XGBoost Performance:")
print(classification_report(y_test, xgb_pred, target_names=['Class 0', 'Class 1']))

# LightGBM 성능 평가를 텍스트로 출력
print("LightGBM Performance:")
print(classification_report(y_test, lgbm_pred, target_names=['Class 0', 'Class 1']))

# CatBoost 성능 평가를 텍스트로 출력
print("CatBoost Performance:")
print(classification_report(y_test, cat_pred, target_names=['Class 0', 'Class 1']))

XGBoost Performance:
              precision    recall  f1-score   support

     Class 0       0.76      0.77      0.77       232
     Class 1       0.76      0.75      0.76       225

    accuracy                           0.76       457
   macro avg       0.76      0.76      0.76       457
weighted avg       0.76      0.76      0.76       457

LightGBM Performance:
              precision    recall  f1-score   support

     Class 0       0.75      0.78      0.77       232
     Class 1       0.77      0.73      0.75       225

    accuracy                           0.76       457
   macro avg       0.76      0.76      0.76       457
weighted avg       0.76      0.76      0.76       457

CatBoost Performance:
              precision    recall  f1-score   support

     Class 0       0.75      0.79      0.77       232
     Class 1       0.77      0.73      0.75       225

    accuracy                           0.76       457
   macro avg       0.76      0.76      0.76       457
weighted 

# 슬라이딩 윈도우 데이터 준비

In [35]:
df_log_target.head()

,Open,High,Low,Volume,moving_average_5,moving_average_10,moving_average_30,moving_average_60,moving_average_120,moving_average_240,target
Date,,,,,,,,,,,
2014-12-26,0.013606,0.007440,0.012992,-1.080436,0.003159,0.000894,-0.002495,-0.001929,-0.000208,-0.000458,1
2014-12-29,0.008745,0.006046,-0.005450,0.323856,0.003696,0.001786,-0.002367,-0.001662,-0.000192,-0.000448,1
2014-12-30,-0.012130,-0.012130,-0.029108,1.088933,0.001638,0.000480,-0.001745,-0.001297,-0.000229,-0.000432,0
2015-01-02,-0.034486,-0.032383,-0.007766,-1.195077,-0.003827,-0.001373,-0.002669,-0.001852,-0.000395,-0.000559,0
2015-01-05,-0.010582,-0.010560,-0.018599,1.353097,-0.006181,-0.001926,-0.002991,-0.001996,-0.000494,-0.000646,0


In [36]:
from sklearn.model_selection import train_test_split

# 데이터 프레임에서'target'을 제외한 나머지를 특징으로 사용
features = df_log_target.drop(['target'], axis=1)

# 10일치 데이터로 구성된 슬라이딩 윈도우 생성
X = []
y = []
window_size = 10

for i in range(len(features) - window_size):
    X.append(features.iloc[i:i+window_size].values.flatten())  # 10일치 데이터를 하나의 배열로 펼침
    y.append(df_log_target.iloc[i + window_size]['target'])  # 해당 윈도우 다음 날의 target

# 배열로 변환
X = np.array(X)
y = np.array(y)

# 데이터 세트 분할: 훈련 세트 80%, 테스트 세트 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1818, 100), (455, 100), (1818,), (455,))

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 랜덤 포레스트 모델 생성 및 훈련
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = rf_model.predict(X_test)

# 예측 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.810989010989011

In [38]:
# XGBoost 모델
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_cm = confusion_matrix(y_test, xgb_pred)

In [39]:
# LightGBM 모델
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_pred = lgbm_model.predict(X_test)
lgbm_cm = confusion_matrix(y_test, lgbm_pred)

[LightGBM] [Info] Number of positive: 886, number of negative: 932
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 1818, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487349 -> initscore=-0.050616
[LightGBM] [Info] Start training from score -0.050616


In [40]:
# CatBoost 모델
cat_model = CatBoostClassifier(verbose=0)  # verbose=0 to avoid too much logging
cat_model.fit(X_train, y_train)
cat_pred = cat_model.predict(X_test)
cat_cm = confusion_matrix(y_test, cat_pred)

In [41]:
xgb_cm, lgbm_cm, cat_cm

(array([[195,  35],
        [ 35, 190]], dtype=int64),
 array([[195,  35],
        [ 36, 189]], dtype=int64),
 array([[197,  33],
        [ 37, 188]], dtype=int64))

In [42]:
from sklearn.metrics import classification_report

# XGBoost 성능 평가
report_xgb = classification_report(y_test, xgb_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# LightGBM 성능 평가
report_lgb = classification_report(y_test, lgbm_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# CatBoost 성능 평가
report_cb = classification_report(y_test, cat_pred, target_names=['Class 0', 'Class 1'], output_dict=True)

# 결과 출력
(report_xgb, report_lgb, report_cb)

({'Class 0': {'precision': 0.8478260869565217,
   'recall': 0.8478260869565217,
   'f1-score': 0.8478260869565218,
   'support': 230},
  'Class 1': {'precision': 0.8444444444444444,
   'recall': 0.8444444444444444,
   'f1-score': 0.8444444444444444,
   'support': 225},
  'accuracy': 0.8461538461538461,
  'macro avg': {'precision': 0.846135265700483,
   'recall': 0.846135265700483,
   'f1-score': 0.8461352657004831,
   'support': 455},
  'weighted avg': {'precision': 0.8461538461538461,
   'recall': 0.8461538461538461,
   'f1-score': 0.8461538461538461,
   'support': 455}},
 {'Class 0': {'precision': 0.8441558441558441,
   'recall': 0.8478260869565217,
   'f1-score': 0.8459869848156182,
   'support': 230},
  'Class 1': {'precision': 0.84375,
   'recall': 0.84,
   'f1-score': 0.8418708240534521,
   'support': 225},
  'accuracy': 0.843956043956044,
  'macro avg': {'precision': 0.843952922077922,
   'recall': 0.8439130434782609,
   'f1-score': 0.8439289044345352,
   'support': 455},
  'wei

In [43]:
# XGBoost 성능 평가를 텍스트로 출력
print("XGBoost Performance:")
print(classification_report(y_test, xgb_pred, target_names=['Class 0', 'Class 1']))

# LightGBM 성능 평가를 텍스트로 출력
print("LightGBM Performance:")
print(classification_report(y_test, lgbm_pred, target_names=['Class 0', 'Class 1']))

# CatBoost 성능 평가를 텍스트로 출력
print("CatBoost Performance:")
print(classification_report(y_test, cat_pred, target_names=['Class 0', 'Class 1']))

XGBoost Performance:
              precision    recall  f1-score   support

     Class 0       0.85      0.85      0.85       230
     Class 1       0.84      0.84      0.84       225

    accuracy                           0.85       455
   macro avg       0.85      0.85      0.85       455
weighted avg       0.85      0.85      0.85       455

LightGBM Performance:
              precision    recall  f1-score   support

     Class 0       0.84      0.85      0.85       230
     Class 1       0.84      0.84      0.84       225

    accuracy                           0.84       455
   macro avg       0.84      0.84      0.84       455
weighted avg       0.84      0.84      0.84       455

CatBoost Performance:
              precision    recall  f1-score   support

     Class 0       0.84      0.86      0.85       230
     Class 1       0.85      0.84      0.84       225

    accuracy                           0.85       455
   macro avg       0.85      0.85      0.85       455
weighted 